In [39]:
import requests
import contextlib

import pandas as pd
import geopandas as gpd
import pathlib

import networkx as nx

In [28]:
data_dir = pathlib.Path('~/data/euris').expanduser()

In [22]:
latest_files_url = 'https://www.eurisportal.eu/AWFImportData/api/ExportFile/GetLatestFiles'

In [23]:
country_codes = ['nl', 'be', 'de', 'lu', 'fr', 'at', 'ch']
all_rows = []
for country_code in country_codes:
    resp = requests.get(latest_files_url, params=dict(countryCode=country_code))
    rows = resp.json()
    all_rows.extend(rows)
files_df = pd.DataFrame(all_rows)


In [34]:
export_file_url = 'https://www.eurisportal.eu/AWFImportData/api/ExportFile/DownloadFile'

def request_zip(row):
    resp = requests.get(export_file_url, params=dict(fileName=row['name']))
    zip_path = data_dir / row['name']
    with open(zip_path, 'wb') as f:
        f.write(resp.content)
    with contextlib.chdir(data_dir):
        !unzip {zip_path.name}
# files_df.apply(request_zip, axis=1);

In [38]:
edges_paths = list(data_dir.glob('FairwaySection*.geojson'))
nodes_paths = list(data_dir.glob('Node*.geojson'))

In [41]:
edges_gdfs = [gpd.read_file(edges_path) for edges_path in edges_paths]
nodes_gdfs = [gpd.read_file(nodes_path) for nodes_path in nodes_paths]

In [49]:
edges_gdf = pd.concat(edges_gdfs)
nodes_gdf = pd.concat(nodes_gdfs)

edges_gdf.to_file(data_dir / 'edges.geojson')
nodes_gdf.to_file(data_dir / 'nodes.geojson')

/tmp/ipykernel_2141026/489402768.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  edges_gdf = pd.concat(edges_gdfs)
